In [1]:
import os

homeDIR = "/usr/local/AvitoML/data/"
#homeDIR = "C:/projects/AvitoML/data/"

os.environ["MKL_THREADING_LAYER"] = "GNU"
import numpy as np
import random
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.metrics import accuracy_score
from nltk.stem import SnowballStemmer
import pymorphy2
from gensim.models.keyedvectors import KeyedVectors
import json
import re
from sklearn.model_selection import train_test_split
#from n05_train_xgb import preproc

morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer('russian')
skip_words = []

SEED = 1369
random.seed(1369)
np.random.seed(1369)

MAX_SEQUENCE_LENGTH = 384
MAX_NB_WORDS = 7040
EMBEDDING_DIM = 300
NUM_CLASS = 54

SAMPLE_SIZE = 400000
print(homeDIR)

Using TensorFlow backend.
/home/azathot/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/usr/local/AvitoML/data/


In [11]:
data = pd.read_csv(homeDIR+'proc_data.csv')
#print(data.head())
print(data.shape) ##489517

(489517, 12)


In [15]:
x = np.array(data['preprocessed1Title'][:SAMPLE_SIZE].tolist())
y = np.array(data['category_id'][:SAMPLE_SIZE])
print(x.shape, y.shape)
print(MAX_SEQUENCE_LENGTH)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens')

data_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels_categ = to_categorical(y)
print('Shape of data tensor:', data_seq.shape)
print('Shape of label tensor:', labels_categ.shape)

    


(400000,) (400000,)
384
Found 113775 unique tokens
Shape of data tensor: (400000, 384)
Shape of label tensor: (400000, 54)


In [22]:
print(type(word_index))
for word, i in word_index.items():
    if i < MAX_NB_WORDS:
        print(word,i)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            print(embedding_vector)
            break        

<class 'dict'>
для 1
продавать 2
новый 3
с 4
на 5
в 6
и 7
2 8
samsung 9
5 10
sony 11
детский 12
3 13
iphone 14
1 15
4 16
из 17
платье 18
galaxy 19
чехол 20
дверь 21
стол 22
коляска 23
монитор 24
xperia 25
nokia 26
6 27
машина 28
часы 29
стиральный 30
телевизор 31
б 32
у 33
гитара 34
набор 35
шкаф 36
кровать 37
ноутбук 38
10 39
женский 40
игра 41
сумка 42
диван 43
комплект 44
asus 45
телефон 46
8 47
по 48
р 49
5s 50
год 51
люстра 52
конек 53
16gb 54
s 55
компьютер 56
ботинок 57
монета 58
колонка 59
lg 60
стенка 61
стул 62
lumia 63
mini 64
белый 65
светильник 66
xbox 67
сапог 68
состояние 69
кроватка 70
ссср 71
7 72
тур 73
от 74
360 75
кольцо 76
блок 77
кормление 78
ps4 79
ipad 80
роутер 81
зимний 82
playstation 83
книга 84
16 85
gb 86
fi 87
ps3 88
wi 89
черный 90
0 91
игровой 92
см 93
туфля 94
продаваться 95
куртка 96
black 97
кресло 98
15 99
стульчик 100
лампа 101
ручной 102
4s 103
картина 104
acer 105
отличный 106
планшет 107
г 108
межкомнатный 109
link 110
стекло 111
philips 112
обме

вега 2246
арсенал 2247
коктейльный 2248
special 2249
армейский 2250
брошь 2251
резинка 2252
тест 2253
няня 2254
battlefront 2255
спица 2256
бытовой 2257
багажник 2258
куб 2259
волна 2260
работать 2261
сб 2262
щетка 2263
рубашка 2264
канал 2265
слуховой 2266
фляжка 2267
птица 2268
midi 2269
элемент 2270
ярусный 2271
500гб 2272
об 2273
филипс 2274
навесной 2275
j1 2276
distortion 2277
ультрабук 2278
любимый 2279
dogs 2280
my 2281
жаккард 2282
be 2283
2гб 2284
футболка 2285
акция 2286
после 2287
cdj 2288
model 2289
ковровый 2290
платяной 2291
hr 2292
ламбрекен 2293
ватрушка 2294
краснодар 2295
fullhd 2296
iv 2297
сделать 2298
шкафчик 2299
br 2300
g3 2301
антиквариат 2302
путешествие 2303
анапа 2304
профиль 2305
пассивный 2306
северный 2307
выходной 2308
ко 2309
y 2310
batman 2311
сова 2312
освещение 2313
turbo 2314
односпальный 2315
e4 2316
atemi 2317
5мм 2318
рожковый 2319
элегантный 2320
франк 2321
уникальный 2322
принт 2323
санкт 2324
портьера 2325
radiotehnika 2326
мотор 2327
ленин 23

индонезия 4960
говорить 4961
фантьеты 4962
вкус 4963
holiday 4964
рецепт 4965
soundking 4966
elitech 4967
палубный 4968
leg 4969
сольфеджио 4970
сигвь 4971
pr 4972
барс 4973
который 4974
pepe 4975
пейзаж 4976
помпон 4977
рольставня 4978
major 4979
ра 4980
куча 4981
дополнительный 4982
ginzzu 4983
round 4984
мария 4985
class 4986
ужин 4987
helicon 4988
пеленание 4989
tascam 4990
лка 4991
sync 4992
дания 4993
выход 4994
980 4995
громкоговоритель 4996
переносной 4997
печать 4998
moon 4999
усилок 5000
динамический 5001
invotone 5002
винить 5003
2мм 5004
подсумок 5005
аврора 5006
паста 5007
развитие 5008
lab 5009
хохлома 5010
ловец 5011
ответ 5012
discovery 5013
паек 5014
base 5015
gx53 5016
comp 5017
пс 5018
воскоплав 5019
губа 5020
d2 5021
308 5022
30w 5023
1964 5024
дамский 5025
5x2 5026
грань 5027
humminbird 5028
лягушка 5029
1958 5030
манга 5031
рулон 5032
песцовый 5033
601 5034
plano 5035
sholl 5036
дневник 5037
проволока 5038
ddr2 5039
bear 5040
точный 5041
его 5042
разноцветный 5043

In [5]:
word2vec_model = KeyedVectors.load_word2vec_format(homeDIR+'ruscorpora_upos_skipgram_300_5_2018.vec', binary=False)


In [23]:

embeddings_index = {}
for word in word2vec_model.vocab:
    embeddings_index[word] = word2vec_model[word]
print('Found %s word vectors.' % len(embeddings_index))
del word2vec_model

embedding_matrix = np.zeros((MAX_NB_WORDS + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

embedding_layer = Embedding(MAX_NB_WORDS + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

Found 195071 word vectors.
(7041, 300)


In [36]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = LSTM(256, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
x = Dense(256, activation='relu')(x)
preds = Dense(NUM_CLASS, activation='softmax')(x)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
json_string = model.to_json()
with open(homeDIR+'lstm.json', 'w') as outfile:
    json.dump(json_string, outfile)

In [37]:
from sklearn.model_selection import StratifiedKFold
x = data_seq 
y = labels_categ 
#word_index 

embedding_laye = embedding_layer
mod = model
mod.summary()

Wsave = mod.get_weights()

batches = [16, 32, 64]
iterations = [5, 5, 5]
accs = []

#skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
fold = 0


mod.set_weights(Wsave)
model_name = f"lstm1_{fold}.h5"


x_trn, x_val, y_trn, y_val = train_test_split(x, y, random_state=666, stratify=y, test_size=0.20, shuffle=True)
print(x_trn.shape, x_val.shape)

b_score = 0.0
for batch_size, it in zip(batches, iterations):
    for i in range(it):
        mod.fit(x_trn, y_trn, batch_size=batch_size, epochs=1, shuffle=True,
                verbose=1, validation_data=(x_val, y_val))

        y_prd = mod.predict(x_val, batch_size=32)
        score = accuracy_score(np.argmax(y_val, axis=1), np.argmax(y_prd, axis=1), )

        print(score)
        if score > b_score:
            b_score = score
            print('ololo')
            mod.save_weights(model_name, overwrite=True)
            y_prd = mod.predict(x_val, batch_size=32)
            np.savez(homeDIR+'lstm_{fold}', y_prd=y_prd, y_val=y_val)

    mod.load_weights(model_name)

mod.load_weights(model_name)
y_prd = mod.predict(x_val, batch_size=32)

np.savez(homeDIR+'lstm_{fold}', y_prd=y_prd, y_val=y_val)

y_prd = np.argmax(y_prd, axis=1)
accs.append(accuracy_score(np.argmax(y_val, axis=1), y_prd))
print(f'fold {fold}: accuracy: {accs[-1]}')

print(f'avg acc: {np.mean(accs)}, std: {np.std(accs)}')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 384)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 384, 300)          2112300   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 54)                13878     
Total params: 2,762,338
Trainable params: 2,762,338
Non-trainable params: 0
_________________________________________________________________
(320000, 384) (80000, 384)
Train on 320000 samples, validate on 80000 samples
Epoch 1/1
320000/320000 [=============================